In [ ]:
import json
import os
import time
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, ImageFileCreateBatch, Region, CustomVisionErrorException
from msrest.authentication import ApiKeyCredentials

# Load the JSON file
json_file_path = "C:\\Users\\feren\\Downloads\\nuimages_output\\train\\labels.json"
with open(json_file_path, "r") as file:
    annotations = json.load(file)


In [ ]:
# Define class mapping
class_mapping = {
    "car": ["automobile", "taxi", "vehicle", "suv", "jeep", "sedan", "van", "land vehicle", "vehicle.car", "vehicle.emergency.police"],
    "truck": ["truck", "lorry", "bus", "shuttle bus", "pickup truck", "vehicle.truck", "vehicle.bus.bendy", "vehicle.bus.rigid", "vehicle.trailer", "vehicle.construction", "vehicle.emergency.ambulance"],
    "person": ["person", "pedestrian", "human", 
               "human.pedestrian.adult", "human.pedestrian.child", "human.pedestrian.construction_worker", 
               "human.pedestrian.personal_mobility", "human.pedestrian.police_officer", "human.pedestrian.stroller", 
               "human.pedestrian.wheelchair"],
    "biker": ["bicycle", "bike", "biker", "motorcycle", "motorbike", "vehicle.bicycle", "vehicle.motorcycle"],
    # Add more mappings as needed
}

In [ ]:
# Reverse class mapping for quick lookup
reverse_class_mapping = {}
for new_class, old_classes in class_mapping.items():
    for old_class in old_classes:
        reverse_class_mapping[old_class] = new_class

# Filter and map annotations
filtered_annotations = []
for annotation in annotations:
    filtered_objects = []
    for obj in annotation["objects"]:
        old_class_name = obj["class_name"]
        if old_class_name in reverse_class_mapping:
            new_class_name = reverse_class_mapping[old_class_name]
            obj["class_name"] = new_class_name
            filtered_objects.append(obj)
    if filtered_objects:
        annotation["objects"] = filtered_objects
        filtered_annotations.append(annotation)


In [ ]:
# Get existing tags
tags = trainer.get_tags(project_id)
tag_dict = {tag.name: tag.id for tag in tags}

# Create tags for all class names in the filtered annotations if they don't already exist
class_names = {obj["class_name"] for annotation in filtered_annotations for obj in annotation["objects"]}

In [ ]:
def create_tag_with_retry(trainer, project_id, class_name, retries=5, backoff_factor=2):
    for attempt in range(retries):
        try:
            return trainer.create_tag(project_id, class_name).id
        except CustomVisionErrorException as e:
            if "Too Many Requests" in str(e):
                wait_time = backoff_factor ** attempt
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                raise
    raise Exception(f"Failed to create tag '{class_name}' after {retries} retries.")

for class_name in class_names:
    if class_name not in tag_dict:
        tag_dict[class_name] = create_tag_with_retry(trainer, project_id, class_name)



In [ ]:
# Prepare image entries
image_folder = "C:\\Users\\feren\\Downloads\\nuimages_output\\train"
image_entries = []

In [ ]:
for annotation in annotations:
    image_filename = annotation["image_id"]
    image_path = os.path.join(image_folder, image_filename)
    
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
        
        image_regions = [
            Region(
                tag_id=tag_dict[obj["class_name"]],
                left=obj["bbox"][0],
                top=obj["bbox"][1],
                width=obj["bbox"][2],
                height=obj["bbox"][3]
            )
            for obj in annotation["objects"]
        ]
        
        image_entries.append(ImageFileCreateEntry(name=image_filename, contents=image_data, regions=image_regions))



In [ ]:
# Upload images in batches
batch_size = 64
for i in range(0, len(image_entries), batch_size):
    batch = ImageFileCreateBatch(images=image_entries[i:i + batch_size])
    try:
        upload_result = trainer.create_images_from_files(project_id, batch)
        if not upload_result.is_batch_successful:
            print("Image batch upload failed.")
            for image in upload_result.images:
                if not image.status == "OK":
                    print(f"Image {image.source_url} upload failed: {image.status}")
    except CustomVisionErrorException as e:
        if "Too Many Requests" in str(e):
            print("Rate limit exceeded while uploading images. Retrying after a delay...")
            time.sleep(60)  # Wait for a minute before retrying
            batch = ImageFileCreateBatch(images=image_entries[i:i + batch_size])
            upload_result = trainer.create_images_from_files(project_id, batch)
            if not upload_result.is_batch_successful:
                print("Image batch upload failed on retry.")
                for image in upload_result.images:
                    if not image.status == "OK":
                        print(f"Image {image.source_url} upload failed: {image.status}")